In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
df = pd.read_csv('judge-1377884607_tweet_product_company.csv', encoding='ISO-8859-1')

In [3]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [5]:
df.shape

(9093, 3)

In [6]:
def clean_text(text):
    # Check if the text is a string
    if not isinstance(text, str):
        return ''  # Return empty string if text is not a string
    # Remove URLs items
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    text = re.sub(r'\@\w+|\#','', text)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters and numbers
    text = re.sub(r"[^a-zA-Z']", ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text

# Clean the tweet_text column
df['cleaned_tweet_text'] = df['tweet_text'].apply(clean_text)

In [7]:
df['cleaned_tweet_text'].head(30)

0     g iphone hrs tweeting rise austin dead need up...
1     know awesome ipad iphone app likely appreciate...
2                              wait ipad also sale sxsw
3     hope year's festival crashy year's iphone app ...
4     great stuff fri sxsw marissa mayer google tim ...
5     new ipad apps speechtherapy communication show...
6                                                      
7     sxsw starting ctia around corner googleio hop ...
8     beautifully smart simple idea rt wrote hollerg...
9     counting days sxsw plus strong canadian dollar...
10    excited meet sxsw show sprint galaxy still run...
11    find amp start impromptu parties sxsw can't wa...
12    foursquare ups game time sxsw still prefer far...
13    gotta love sxsw google calendar featuring top ...
14                                  great sxsw ipad app
15          haha awesomely rad ipad app hollergram sxsw
16           holler gram ipad itunes app store via sxsw
17    noticed dst coming weekend many iphone use

In [8]:
pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_columns')



In [9]:
df['cleaned_tweet_text'].head(10)

0    g iphone hrs tweeting rise austin dead need up...
1    know awesome ipad iphone app likely appreciate...
2                             wait ipad also sale sxsw
3    hope year's festival crashy year's iphone app ...
4    great stuff fri sxsw marissa mayer google tim ...
5    new ipad apps speechtherapy communication show...
6                                                     
7    sxsw starting ctia around corner googleio hop ...
8    beautifully smart simple idea rt wrote hollerg...
9    counting days sxsw plus strong canadian dollar...
Name: cleaned_tweet_text, dtype: object

In [10]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


lemmatizer = WordNetLemmatizer()

 
def lemmatize_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Rejoin lemmatized tokens into a string
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text

# Example text
example_text = "Fall Out Boy Rules."

# Lemmatize the example text
lemmatized_text = lemmatize_text('cleaned_tweet_text')
print(lemmatized_text)


cleaned_tweet_text


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer
count_vect = CountVectorizer()

# Fit and transform the lemmatized text
# This step converts your text data into a matrix of token counts
X_counts = count_vect.fit_transform(df['lemmatized_text'])

# X_counts now contains the count vectorized form of your text data
# You can convert this to a DataFrame if you want to see the token counts
count_vect_df = pd.DataFrame(X_counts.toarray(), columns=count_vect.get_feature_names_out())

# Display the first few rows of the DataFrame
print(count_vect_df.head())


KeyError: 'lemmatized_text'